In [3]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *


# Loading your IBM Quantum account(s)
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options
service = QiskitRuntimeService(channel="ibm_quantum")

# Define backends
use_real = False

if use_real:
    backend = service.backend('ibm_wellington')
else:
    backend = service.backend('ibmq_qasm_simulator')

In [4]:
# Operators from Qiskit
from qiskit.quantum_info import SparsePauliOp

H_problem = SparsePauliOp.from_list([('YZ', 0.39793742484318045),
                                     ('ZI', -0.39793742484318045),
                                     ('ZZ', -0.01128010425623538),
                                     ('XX', 0.18093119978423156)])

print (H_problem)

SparsePauliOp(['YZ', 'ZI', 'ZZ', 'XX'],
              coeffs=[ 0.39793742+0.j, -0.39793742+0.j, -0.0112801 +0.j,  0.1809312 +0.j])


In [5]:
H_mat = H_problem.to_matrix()
np.linalg.eigvalsh(H_mat)

array([-0.70278562, -0.45302961,  0.45302961,  0.70278562])

In [6]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(2)

ansatz.draw()

»
q_0: »
     »
q_1: »
     »
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                     ├
«     │  EfficientSU2(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7],θ[8],θ[9],θ[10],θ[11],θ[12],θ[13],θ[14],θ[15]) │
«q_1: ┤1                                                                                                     ├
«     └──────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [7]:
num_params = ansatz.num_parameters
params = 2*np.pi*np.random.random(num_params)

In [8]:
def build_cost_function(estimator, shots=int(1e4)):
    """ Wrapper that allows for setting estimator for cost function
    """
    def vqe_function(params, shots=shots):
        """Return estimate of energy from estimator

        Parameters:
            params (ndarray): Array of ansatz parameters
            estimator (Estimator): Estimator instance
            shots (int): Number of shots to take

        Returns:
            float: Energy estimate
        """
        energy = estimator.run(ansatz, H_problem, shots=shots,
                               parameter_values=params).result().values[0]
        return energy
    
    return vqe_function

In [15]:
# Add path for helpers
import sys  
sys.path.insert(0, '.')

from spsa import *

sys.path.remove('.')

optim = SPSA()

In [ ]:
session = Session(service=service, backend=backend)
estimator = Estimator(session=session)

cost_func = build_cost_function(estimator)

res = optim.minimize(cost_func, x0=params)

In [ ]:
print(res)